In [60]:
from glob import iglob as list_paths


def load_text(path):
    with open(path, encoding='utf-8') as file:
        return file.read()


texts = [
    load_text(_)
    for _ in list_paths('3_fav_ex/*.txt')
]

In [2]:
from natasha import (
    NamesExtractor,
    SimpleNamesExtractor,
    PersonExtractor,

    LocationExtractor,
    AddressExtractor,

    OrganisationExtractor,

    DatesExtractor,

    MoneyExtractor,
    MoneyRateExtractor,
    MoneyRangeExtractor,
)

from natasha.markup import (
    show_markup_notebook as show_markup,
    format_json
)

In [49]:
from yargy.token import Span

def unite_spans(spans):
    if not spans:
        return spans
    spans = sorted(spans, key=lambda x: (x.start, x.stop))
    new_spans = []
    beg, end = spans[0].start, spans[0].stop
#     prev = spans[0]
    for span in spans[1:]:
        if span.start < end < span.stop:
            end = span.stop
        elif span.stop <= end:
            continue
        else:
            new_spans.append(Span(beg, end))
            beg, end = span.start, span.stop
    new_spans.append(Span(beg, end))
    return new_spans

In [67]:
def find_entities(text):
    all_spans = []
    for extr_cls in (
        NamesExtractor,
#         SimpleNamesExtractor,
#         PersonExtractor,
        LocationExtractor,
        AddressExtractor,
        OrganisationExtractor,
        DatesExtractor,
        MoneyExtractor,
        MoneyRateExtractor,
        MoneyRangeExtractor,
    ):
        extractor = extr_cls()
        matches = extractor(text)
        spans = [_.span for _ in matches]
        all_spans.extend(spans)
    return unite_spans(all_spans)


for text in texts:
    all_spans = find_entities(text)
    show_markup(text, all_spans)
    print('\n\n\n')

In [68]:
def find_entities(text):
    all_spans = []
    for extr_cls in (
        NamesExtractor,
#         SimpleNamesExtractor,
#         PersonExtractor,
#         LocationExtractor,
        AddressExtractor,
#         OrganisationExtractor,
        DatesExtractor,
#         MoneyExtractor,
#         MoneyRateExtractor,
#         MoneyRangeExtractor,
    ):
        extractor = extr_cls()
        matches = extractor(text)
        spans = [_.span for _ in matches]
        all_spans.extend(spans)
    return unite_spans(all_spans)


for text in texts:
    all_spans = find_entities(text)
    show_markup(text, all_spans)
    print('\n\n\n')

In [69]:
texts = ['Здравствуйте, Алексей Сергеевич',
         'Здравствуйте, Алексей Сергеевич!',
         'Здравствуйте, Максим Сергеевич!',
         'Санкт-Петербург',
        'наб. Крюкова канала, д. 12, Санкт-Петербург.',
        'Санкт-Петербург, наб. Крюкова канала, д. 12']

for text in texts:
    all_spans = find_entities(text)
    show_markup(text, all_spans)

In [52]:
texts = '''
Так говорила в июле 1805 года известная Анна Павловна Шерер, фрейлина и приближенная императрицы Марии Феодоровны, встречая важного и чиновного князя Василия, первого приехавшего на ее вечер. Анна Павловна кашляла несколько дней, у нее был грипп, как она говорила (грипп был тогда новое слово, употреблявшееся только редкими).
Предлагаю вернуть прежние границы природного парка №71 на Инженерной улице 2. По адресу Алтуфьевское шоссе д.51 (основной вид разрешенного использования: производственная деятельность, склады) размещен МПЗ. Жители требуют незамедлительной остановки МПЗ и его вывода из района
Контакты О нас телефон 7 881 574-10-02 Адрес Республика Карелия,г.Петрозаводск,ул.Маршала Мерецкова, д.8 Б,офис 4 
Благодарственное письмо Хочу поблагодарить учителей моего, теперь уже бывшего, одиннадцатиклассника: Бушуева Вячеслава Владимировича и Бушуеву Веру Константиновну. Они вовлекали сына в интересные внеурочные занятия, связанные с театром и походами.
'''.splitlines()

for text in texts:
    all_spans = find_entities(text)
    show_markup(text, all_spans)